In [1]:
%env CUDA_VISIBLE_DEVICES=0

env: CUDA_VISIBLE_DEVICES=0


In [2]:
#! /usr/bin/env python3
# -*- coding: utf-8 -*-
import os
import gc

import numpy as np
import torch
import torch.backends.cudnn as cudnn
from torch import nn

from dataloader import svhn
from models import ShuffleNetV2
from src import freeze_influence, hessians, selection

device = "cuda" if torch.cuda.is_available() else "cpu"
target_removal_label = 0

In [3]:
def load_net(net, path):
    assert os.path.isfile(path), "Error: no checkpoint file found!"
    checkpoint = torch.load(path)
    net.load_state_dict(checkpoint["net"])
    return net


def save_net(net, path):
    dir, filename = os.path.split(path)
    if not os.path.isdir(dir):
        os.makedirs(dir)

    state = {
        "net": net.state_dict(),
    }
    torch.save(state, path)


def test(net, dataloader, criterion, label, include):
    net.eval()
    with torch.no_grad():
        net_loss = 0
        correct = 0
        total = 0
        num_data = 0
        for _, (inputs, targets) in enumerate(dataloader):
            if include:
                idx = targets == label
            else:
                idx = targets != label
            inputs = inputs[idx]
            targets = targets[idx]
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            net_loss += loss * len(inputs)
            num_data +=  len(inputs)

            total += targets.size(0)
            _, predicted = outputs.max(1)
            correct += predicted.eq(targets).sum().item()

        accuracy = correct / total * 100
        net_loss /= num_data
        return net_loss, accuracy

def influence_test(net, dataloader, criterion, target_label):
    def sample_test(net, criterion, inputs, targets):
        net.eval()
        with torch.no_grad():
            inputs, targets = inputs.to(device), targets.to(device)
            outputs = net(inputs)
            loss = criterion(outputs, targets)
            _, predicted = outputs.max(1)
            correct = predicted.eq(targets).sum().item()

            return loss, correct

    self_loss = 0
    self_correct = 0
    num_self_inputs = 0
    
    exclusive_loss = 0
    exclusive_correct = 0
    num_exclusive_inputs = 0
    
    for _, (inputs, targets) in enumerate(dataloader):
        target_idx = (targets == target_label)
        batch_self_loss, batch_self_correct = sample_test(net, criterion, inputs[target_idx], targets[target_idx])
        batch_exclusive_loss, batch_exclusive_correct = sample_test(net, criterion, inputs[~target_idx], targets[~target_idx])
        
        len_self_batch = len(inputs[target_idx])
        self_loss += batch_self_loss * len_self_batch
        self_correct += batch_self_correct
        num_self_inputs += len_self_batch
        
        len_exclusive_batch = len(inputs[~target_idx])
        exclusive_loss += batch_exclusive_loss * len_exclusive_batch
        exclusive_correct += batch_exclusive_correct
        num_exclusive_inputs += len_exclusive_batch
        
    self_loss /= num_self_inputs
    self_acc = self_correct / num_self_inputs * 100
    exclusive_loss /= num_exclusive_inputs
    exclusive_acc = exclusive_correct / num_exclusive_inputs * 100
    
    return self_loss, self_acc, exclusive_loss, exclusive_acc
        
def projected_influence(net, total_loss, target_loss, index_list, tol, step, max_iter, verbose):
    num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
    full_param_index_list = np.arange(num_param)
    influence = hessians.generalized_influence(
        net, total_loss, target_loss, full_param_index_list, tol=tol, step=step, max_iter=max_iter, verbose=verbose
    )
    return influence[index_list]

def f1_score(self_acc, test_acc):
    self_acc /= 100
    test_acc /= 100
    if self_acc == 1 and test_acc == 0:
        return 0
    return 2 * (1 - self_acc) * test_acc / (1 - self_acc + test_acc)

In [4]:
net = ShuffleNetV2().to(device)
net_name = "ShuffleNetV2"

if device == "cuda":
    cudnn.benchmark = True

net_path = f"checkpoints/tab2/{net_name}/cross_entropy/ckpt_0.0.pth"
net = load_net(net, net_path)

net.eval()
num_param = sum(p.numel() for p in net.parameters() if p.requires_grad)
print(
    f"==> Building {net_name} finished. "
    + f"\n    Number of parameters: {num_param}"
)

criterion = nn.CrossEntropyLoss()

# Data
print("==> Preparing data..")
batch_size = 256
num_workers = 16
num_sample_batch = 1
num_target_sample = 512

data_loader = svhn.SVHNDataLoader(batch_size, num_workers, validation=False)
train_loader, test_loader = data_loader.get_data_loaders()

loss, acc = test(net, test_loader, criterion, 11, False)
print(
    f"Original loss and acc : {loss:.4f}, {acc:.2f}%"
)

==> Building ShuffleNetV2 finished. 
    Number of parameters: 1263854
==> Preparing data..
Using downloaded and verified file: data/train_32x32.mat
Using downloaded and verified file: data/test_32x32.mat
Original loss and acc : 0.1746, 95.40%


In [5]:
verbose = True
num_exp = 10

removal_inputs = list()
removal_targets = list()
for batch_idx, (inputs, targets) in enumerate(test_loader):
    idx = targets == target_removal_label
    removal_inputs.append(inputs[idx])
    removal_targets.append(targets[idx])
removal_inputs = torch.cat(removal_inputs)
removal_targets = torch.cat(removal_targets)

ratio_list = [.1, .3, .5]
result_list_GIF = []
result_list_FIF = []
result_list_PIF = []

tol = 1e-9

for exp_iter in range(num_exp):
    sample_idx = np.random.choice(len(removal_inputs), num_target_sample, replace=False)
    for param_ratio in ratio_list:
        for i in range(3):
            if i == 0:
                if_name = "GIF"
            elif i == 1:
                if_name = "FIF"
            else:
                if_name = "PIF"

            print(f"{if_name} - ratio: {param_ratio*100}%, tol: {tol}")
            # Initialize network
            net = load_net(net, net_path)

            # Compute total loss
            total_loss = 0
            for batch_idx, (inputs, targets) in enumerate(test_loader):
                if batch_idx >= num_sample_batch:
                    break
                idx = targets != target_removal_label
                inputs, targets = inputs[idx], targets[idx]
                inputs, targets = inputs.to(device), targets.to(device)
                outputs = net(inputs)
                total_loss += criterion(outputs, targets)

            # Sampling the target removal data
            sample_removal_inputs = removal_inputs[sample_idx]
            sample_removal_targets = removal_targets[sample_idx]
            
            # Make hooks
            net_parser = selection.TopNGradients(net, param_ratio)
            net_parser.register_hooks()

            # Compute target loss
            target_loss = (
                criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                * len(removal_inputs)
                / (len(train_loader.dataset) - len(removal_inputs))
            )
            target_loss.backward()
            net_parser.remove_hooks()

            target_loss = (
                criterion(net(sample_removal_inputs.to(device)), sample_removal_targets.to(device))
                * len(removal_inputs)
                / (len(train_loader.dataset) - len(removal_inputs))
            )
            
            # Delete hooks
            index_list = net_parser.get_parameters()
            net_parser.remove_hooks()
            
            if i == 0:
                influence = hessians.generalized_influence(
                    net, total_loss, target_loss, index_list, tol=tol, step=3, max_iter=30, verbose=False
                )
            elif i == 1:
                influence = freeze_influence.freeze_influence(
                    net, total_loss, target_loss, index_list, tol=tol, step=3, max_iter=30, verbose=False
                )
            else:
                influence = projected_influence(
                    net, total_loss, target_loss, index_list, tol=tol, step=3, max_iter=30, verbose=False
                )
                
            del total_loss, target_loss
            gc.collect()
            torch.cuda.empty_cache()
            
            influence *= 0.03 / torch.norm(influence)
                
            scale = 1 if i!=2 else 1 / param_ratio
            score = 0
            best_score = -1
            count = 1
            save_path = (
                f"checkpoints/tab2/{net_name}/{if_name}/{target_removal_label}_{param_ratio}_{exp_iter}.pth"
            )
            while True:
                if score < .85:
                    net_parser.update_network(influence * scale)
                else:
                    net_parser.update_network(influence * scale / 3)

#                 self_loss, self_acc = test(net, test_loader, criterion, target_removal_label, True)
#                 exclusive_loss, exclusive_acc = test(net, test_loader, criterion, target_removal_label, False)
                self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                                                    criterion, target_removal_label)
                score = f1_score(self_acc, exclusive_acc)
                
                if best_score < score:
                    best_result = [exclusive_acc, exclusive_loss, self_acc, self_loss, score]
                    best_score = score
                    save_net(net, save_path)
                    
                if verbose:
                    print(
                    f"{count} - test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} |" +
                    f" self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | score: {score:.7f}",
                    end='\r'
                    )
                
                if exclusive_acc < 80 or self_acc < 0.01 or count >= 200:
                    if i == 0:
                        result_list_GIF += best_result
                    elif i == 1:
                        result_list_FIF += best_result
                    else:
                        result_list_PIF += best_result
                    
                    print(f"test acc: {best_result[0]:2.2f}, test loss: {best_result[1]:.4f} | " +
                          f"self-acc: {best_result[2]:2.2f}%, self loss: {best_result[3]:.4f} | " +
                          f"Score: {best_result[4]:.7f}" + " " * 20) 
                    break
                elif count >= 20 and best_score < 0.2:
                    scale *= 5
                elif count >= 50 and best_score < 0.5:
                    print(f"test acc: {best_result[0]:2.2f}, test loss: {best_result[1]:.4f} | " +
                          f"self-acc: {best_result[2]:2.2f}%, self loss: {best_result[3]:.4f} | " +
                          f"Score: {best_result[4]:.7f}" + " " * 20) 
                    break

                count += 1
                
        print("")

GIF - ratio: 10.0%, tol: 1e-09
test acc: 94.85, test loss: 0.1884 | self-acc: 0.06%, self loss: 4.5439 | Score: 0.9733156                    
FIF - ratio: 10.0%, tol: 1e-09
test acc: 94.79, test loss: 0.1904 | self-acc: 0.06%, self loss: 4.6774 | Score: 0.9729904                    
PIF - ratio: 10.0%, tol: 1e-09
test acc: 94.24, test loss: 0.2120 | self-acc: 0.00%, self loss: 5.7563 | Score: 0.9703239                    

GIF - ratio: 30.0%, tol: 1e-09
test acc: 94.32, test loss: 0.2086 | self-acc: 0.06%, self loss: 5.3048 | Score: 0.9705118                    
FIF - ratio: 30.0%, tol: 1e-09
test acc: 94.39, test loss: 0.2038 | self-acc: 0.23%, self loss: 5.0681 | Score: 0.9700480                    
PIF - ratio: 30.0%, tol: 1e-09
test acc: 94.45, test loss: 0.2014 | self-acc: 0.17%, self loss: 5.0864 | Score: 0.9706452                    

GIF - ratio: 50.0%, tol: 1e-09
test acc: 94.24, test loss: 0.2116 | self-acc: 0.17%, self loss: 5.1708 | Score: 0.9695134                    
FIF 

test acc: 94.76, test loss: 0.1940 | self-acc: 0.17%, self loss: 4.9137 | Score: 0.9722949                    
FIF - ratio: 30.0%, tol: 1e-09
test acc: 94.47, test loss: 0.2028 | self-acc: 0.17%, self loss: 5.1489 | Score: 0.9707322                    
PIF - ratio: 30.0%, tol: 1e-09
test acc: 94.60, test loss: 0.1972 | self-acc: 0.17%, self loss: 4.9542 | Score: 0.9714273                    

GIF - ratio: 50.0%, tol: 1e-09
test acc: 94.33, test loss: 0.2099 | self-acc: 0.23%, self loss: 5.0860 | Score: 0.9697218                    
FIF - ratio: 50.0%, tol: 1e-09
test acc: 94.28, test loss: 0.2107 | self-acc: 0.17%, self loss: 5.2647 | Score: 0.9697312                    
PIF - ratio: 50.0%, tol: 1e-09
test acc: 94.26, test loss: 0.2127 | self-acc: 0.11%, self loss: 5.3540 | Score: 0.9698927                    

GIF - ratio: 10.0%, tol: 1e-09
test acc: 94.61, test loss: 0.1987 | self-acc: 0.17%, self loss: 4.7662 | Score: 0.9715141                    
FIF - ratio: 10.0%, tol: 1e-09
test

In [9]:
for param_ratio in ratio_list:
    for i in range(3):
        if i == 0:
            if_name = "GIF"
        elif i == 1:
            if_name = "FIF"
        else:
            if_name = "PIF"
        print(f"{if_name} - ratio: {param_ratio*100}%, tol: {tol}")
        
        self_loss_list = np.empty(0)
        self_acc_list = np.empty(0)
        exclusive_loss_list = np.empty(0)
        exclusive_acc_list = np.empty(0)
        f1_score_list = np.empty(0)
        
        for exp_iter in range(num_exp):

            load_path = (
                f"checkpoints/tab2/{net_name}/{if_name}/{target_removal_label}_{param_ratio}_{exp_iter}.pth"
            )
            net = ShuffleNetV2().to(device)
            net = load_net(net, load_path)
            self_loss, self_acc, exclusive_loss, exclusive_acc = influence_test(net, test_loader, 
                                                                    criterion, target_removal_label)
            score = f1_score(self_acc, exclusive_acc)
            
            self_loss_list = np.append(self_loss_list, self_loss.detach().cpu().numpy())
            self_acc_list = np.append(self_acc_list, self_acc)
            exclusive_loss_list = np.append(exclusive_loss_list, exclusive_loss.detach().cpu().numpy())
            exclusive_acc_list = np.append(exclusive_acc_list, exclusive_acc)
            f1_score_list = np.append(f1_score_list, score)
            print(
            f"  test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} | " +
            f"self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | score: {score:.4f}",
            end='\r'
            )
            
        mean_self_loss = np.mean(self_loss_list)
        mean_self_acc = np.mean(self_acc_list)
        mean_exclusive_loss = np.mean(exclusive_loss_list)
        mean_exclusive_acc = np.mean(exclusive_acc_list)
        mean_f1_score = np.mean(f1_score_list)
                
        var_self_loss = np.var(self_loss_list)
        var_self_acc = np.var(self_acc_list)
        var_exclusive_loss = np.var(exclusive_loss_list)
        var_exclusive_acc = np.var(exclusive_acc_list)
        var_f1_score = np.var(f1_score_list)

        print(
        f"test acc: {mean_exclusive_acc:2.2f}+-{var_exclusive_acc:2.2f}% " +
        f"test loss: {mean_exclusive_loss:.4f}+-{var_exclusive_loss:.4f} ", end=""
        )
        print(
        f"self-acc: {mean_self_acc:2.2f}+-{var_self_acc:2.2f}% " +
        f"self loss: {mean_self_loss:.4f}+-{var_self_loss:.4f} " +
        f"score: {mean_f1_score:.4f}",
        )

GIF - ratio: 10.0%, tol: 1e-09
test acc: 94.80+-0.05% test loss: 0.1909+-0.0001 self-acc: 0.07+-0.00% self loss: 4.6834+-0.0439 score: 0.9730
FIF - ratio: 10.0%, tol: 1e-09
test acc: 94.74+-0.04% test loss: 0.1923+-0.0001 self-acc: 0.08+-0.00% self loss: 4.7154+-0.0450 score: 0.9726
PIF - ratio: 10.0%, tol: 1e-09
test acc: 94.27+-0.17% test loss: 0.2096+-0.0003 self-acc: 0.00+-0.00% self loss: 5.7399+-0.2117 score: 0.9705
GIF - ratio: 30.0%, tol: 1e-09
test acc: 94.24+-0.11% test loss: 0.2129+-0.0002 self-acc: 0.16+-0.01% self loss: 5.2122+-0.1077 score: 0.9696
FIF - ratio: 30.0%, tol: 1e-09
test acc: 94.22+-0.09% test loss: 0.2137+-0.0002 self-acc: 0.20+-0.01% self loss: 5.2114+-0.0976 score: 0.9693
PIF - ratio: 30.0%, tol: 1e-09
test acc: 94.29+-0.10% test loss: 0.2098+-0.0002 self-acc: 0.18+-0.01% self loss: 5.1855+-0.1280 score: 0.9697
GIF - ratio: 50.0%, tol: 1e-09
test acc: 94.01+-0.13% test loss: 0.2225+-0.0003 self-acc: 0.22+-0.01% self loss: 5.3163+-0.1167 score: 0.9681
FIF - 

In [10]:
retrained_net = ShuffleNetV2().to(device)
net_name = "ShuffleNetV2"
net_path = f"checkpoints/tab2/{net_name}_retrained/cross_entropy/ckpt_0.0_{target_removal_label}.pth"
print(net_path)
retrained_net = load_net(retrained_net, net_path)

self_loss, self_acc = test(retrained_net, test_loader, criterion, target_removal_label, True)
exclusive_loss, exclusive_acc = test(retrained_net, test_loader, criterion, target_removal_label, False)
score = f1_score(self_acc, exclusive_acc)

print(
f"test acc: {exclusive_acc:2.2f}, test loss: {exclusive_loss:.4f} |" +
f" self-acc: {self_acc:2.2f}%, self loss: {self_loss:.4f} | score: {score:.7f}",
)

checkpoints/tab2/ShuffleNetV2_retrained/cross_entropy/ckpt_0.0_0.pth
test acc: 95.47, test loss: 0.1673 | self-acc: 0.00%, self loss: 7.2099 | score: 0.9768304


## 